In [1]:
import itertools
import warnings
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from src.config import BLD, SRC
from src.estimation.gridsearch import (
    get_mask_around_diagonal,
    run_1d_gridsearch,
    run_2d_gridsearch,
)
from src.estimation.msm_criterion import (
    get_index_bundles,
    get_parallelizable_msm_criterion,
)
from src.manfred.shared import hash_array
from src.plotting.msm_plots import plot_estimation_moment, plot_infection_channels
from src.simulation.load_params import load_params
from src.simulation.load_simulation_inputs import load_simulation_inputs

# -----------------------------------------------------------------------------------

DEBUG = False
prefix = "delta"

# -----------------------------------------------------------------------------------


FALL_SIM_START = pd.Timestamp("2021-04-11")  # spring end is 2021-04-10
FALL_SIM_END = pd.Timestamp("2021-07-01")

SPRING_SIM_START = pd.Timestamp("2021-07-02")  # summer end. not used
SPRING_SIM_END = pd.Timestamp("2021-08-26")  # not used

if DEBUG:
    FALL_SIM_END = FALL_SIM_START + pd.Timedelta(days=3)
    SPRING_SIM_END = SPRING_SIM_START + pd.Timedelta(days=3)

warnings.filterwarnings(
    "ignore", message="indexing past lexsort depth may impact performance."
)

# %load_ext snakeviz

# Load the simulation inputs

In [2]:
params = load_params("baseline")

pmsm = get_parallelizable_msm_criterion(
    prefix=prefix,
    fall_start_date=FALL_SIM_START,
    fall_end_date=FALL_SIM_END,
    spring_start_date=SPRING_SIM_START,
    spring_end_date=SPRING_SIM_END,
    mode="fall",
    debug=DEBUG,
)

# Modify Params

In [3]:
delta_params = [("events", "delta_cases_per_100_000", "2021-05-15")]
rapid_reduc_params = [
    ("rapid_test_demand", "low_incidence_factor", "other_demand"),
    ("rapid_test_demand", "low_incidence_factor", "worker_demand"),
]

free_params = delta_params + rapid_reduc_params

# Set some free parameters-----------------------------------------------------------

# params.loc[delta_params] = 0.1

# -----------------------------------------------------------------------------------

params.loc[free_params]

value
category          subcategory             name                
events            delta_cases_per_100_000 2021-05-15      0.10
rapid_test_demand low_incidence_factor    other_demand    0.25
                                          worker_demand   0.25

# Specify Grid

In [4]:
dimensions = "1d"

n_gridpoints = 10 if not DEBUG else 1
loc1 = delta_params

# -----------------------------------------------------------------------------------

gridspec1 = (0.01, 0.12, n_gridpoints)
OUT_PATH = BLD / "simulation" / f"calibration_{prefix}_001_to_012_8"
OUT_PATH.mkdir(parents=True)

# -----------------------------------------------------------------------------------

loc2 = None
gridspec2 = ("low", "high", n_gridpoints)

n_seeds = 20 if not DEBUG else 1
n_cores = 40 if not DEBUG else 1
mask = None

In [5]:
initial_states_path = (
    str(BLD / "simulations" / "last_states") + "/verify_spring_baseline_{seed}.pkl"
)

# Run estimation

In [ ]:
start_time = pd.Timestamp.now()
print(start_time)

if dimensions == "2d":
    results, grid, best_index, fig = run_2d_gridsearch(
        func=pmsm,
        params=params,
        loc1=loc1,
        gridspec1=gridspec1,
        loc2=loc2,
        gridspec2=gridspec2,
        n_seeds=n_seeds,
        n_cores=n_cores,
        mask=mask,
        initial_states_path=initial_states_path,
    )
else:
    results, grid, best_index, fig = run_1d_gridsearch(
        func=pmsm,
        params=params,
        loc=loc1,
        gridspec=gridspec1,
        n_seeds=n_seeds,
        n_cores=n_cores,
        initial_states_path=initial_states_path,
    )

end_time = pd.Timestamp.now()

2021-09-01 05:50:33.271594


Resume the simulation...
2021-04-11:   0%|          | 0/82 [00:00<?, ?it/s]Resume the simulation...
Resume the simulation...
2021-04-26:  18%|█▊        | 15/82 [03:36<17:17, 15.49s/it]

In [ ]:
print(end_time - start_time)

In [ ]:
pd.to_pickle(results, OUT_PATH / "delta_results.pkl")

In [ ]:
import os

os.listdir("/home/klara/disk/sid-germany/bld/simulations/last_states")

# Plot Criterion Values

In [ ]:
fig
fig.savefig(OUT_PATH / "x_to_criterion.pdf")

# Plot Delta Shares

In [ ]:
for g, res in zip(grid, results):
    print(g)
    fig = plot_estimation_moment(res, "aggregated_delta_share")
    fig.savefig(OUT_PATH / f"delta_share_{g}.pdf")

# Plot Case Numbers

In [ ]:
ix = best_index
fig = plot_estimation_moment(results[ix], "aggregated_infections_not_log")
fig.savefig(OUT_PATH / f"cases_best_{grid[ix]}.pdf")

In [ ]:
plot_estimation_moment(results[ix], "infections_by_age_group")

In [ ]:
results[0][0].keys()

In [ ]:
results[0][0]["simulated_moments"].keys()